# Run Predictions - <font color = "red">0.0.categ_v1</font>

In [1]:
import json
import urllib.request
import os
os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

from src.ml.ctw import MyCTW
from src.dao import csv_dao
from src.entity.stop_region import StopRegionGroup, sr_row_to_stop_region
from src.exceptions import exceptions
from src.utils.others import remove_list_elements
from src.exceptions.exceptions import NoCategoryMatched

from src.experiments.ctw_eval import evaluation_ctw_single_partition_light_mem

from src.experiments.markov_eval import evaluation_markov_k_fold_light_mem, evaluation_markov_all_users_vs_one_light_mem, evaluation_markov_single_partition_light_mem, evaluation_markov_cluster_light_mem

from src.taxonomy.category_mapping import CategoryMapper

from src.exceptions.exceptions import NoCategoryMatched, NotValidTypes

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

working dir /home/tales/dev/master/mdc_analysis


## Loading Data

In [2]:
def get_users_tags_sequence():
    url = "http://127.0.0.1:5000/stop_regions_group"
    response = urllib.request.urlopen(url)
    return json.loads(response.read())

def load_users_tags_sequence():
    with open('outputs/users_tags_sequence.json') as json_file:
        return json.load(json_file)
    
users_tags_sequence = load_users_tags_sequence()

In [3]:
def clean_sequence(sequence):
    new_sequence = []
    
    for tags in sequence:
        if "parking" in tags:
            continue
        else:
            new_sequence.append(tags)

    return new_sequence

In [4]:
categ_mapper = CategoryMapper()

users_categ_sequence = {}
users_categ_sequence_not_list = {}

n=0
for user_id in list(users_tags_sequence.keys()):
    n+=1
    print("n:", n, "user_id:", user_id)
    
    users_tags_sequence[user_id] = clean_sequence(users_tags_sequence[user_id])
    
    if len( remove_list_elements(users_tags_sequence[user_id], elements=[[]]) ) < 8:
        continue 
    
    categ_sequence = []

    for tags in users_tags_sequence[user_id]:
        if tags == ["WORK"] or tags == ["HOME"]:
            categ_sequence.append(tags)

        else:
            try:
#                 print("tags:", tags)
                categ = categ_mapper.map_categ(tags, method="most_specific")
                categ_sequence.append([categ])

            except NotValidTypes:
                pass
            
            except NoCategoryMatched:
                categ_sequence.append(["NoCategoryMatched"]) 

    users_categ_sequence[user_id] = categ_sequence
    users_categ_sequence_not_list[user_id] = [categ[0] for categ in categ_sequence]
    

n: 1 user_id: 6189
n: 2 user_id: 5936
n: 3 user_id: 6087
n: 4 user_id: 5973
n: 5 user_id: 6085
n: 6 user_id: 6074
n: 7 user_id: 6012
n: 8 user_id: 5982
n: 9 user_id: 5948
n: 10 user_id: 5974
n: 11 user_id: 6090
n: 12 user_id: 6199
n: 13 user_id: 6068
n: 14 user_id: 6024
n: 15 user_id: 5976
n: 16 user_id: 6094
n: 17 user_id: 5941
n: 18 user_id: 5995
n: 19 user_id: 5962
n: 20 user_id: 6093
n: 21 user_id: 6033
n: 22 user_id: 6079
n: 23 user_id: 6038
n: 24 user_id: 6175
n: 25 user_id: 6042
n: 26 user_id: 5924
n: 27 user_id: 6083
n: 28 user_id: 6178
n: 29 user_id: 5958
n: 30 user_id: 6086
n: 31 user_id: 6100
n: 32 user_id: 5945
n: 33 user_id: 5925
n: 34 user_id: 5935
n: 35 user_id: 6172
n: 36 user_id: 5955
n: 37 user_id: 6073
n: 38 user_id: 5980
n: 39 user_id: 6010
n: 40 user_id: 5993
n: 41 user_id: 6037
n: 42 user_id: 5979
n: 43 user_id: 5966
n: 44 user_id: 5985
n: 45 user_id: 5967
n: 46 user_id: 5970
n: 47 user_id: 6169
n: 48 user_id: 6188
n: 49 user_id: 6097
n: 50 user_id: 6096
n: 51 use

In [6]:
# users_tags_sequence = {}

# users = os.listdir("outputs/stop_regions/")
# users.reverse()

# for user_id in users:
#     print("Loading user {} data".format(user_id))
#     users_tags_sequence[user_id] = StopRegionGroup(csv_dao.stop_region_sequence(user_id), 
#                                                     agglutinate_stop_regions=True).sequence_stop_region_tags()["tag"].tolist()

## CTW

#### SINGLE PARTITION - Same train and test data for each same user

In [5]:
skipped_users = []

depths = [2, 3]

predict_choice_methods = ["random_choice", "most_likely", "random_dummy"]

n=0
for user_id in users_categ_sequence.keys():
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    
    for depth in depths:
        
        for predict_choice_method in predict_choice_methods:
    
            tags_sequence = [categ[0] for categ in users_categ_sequence[user_id]]
            
            if len( remove_list_elements(tags_sequence, elements=[[]]) ) < 8:
                skipped_users.append(user_id)
                continue    

            try:
                evaluation_ctw_single_partition_light_mem(tags_sequence=tags_sequence, 
                                                          user_id=user_id, 
                                                          input_data_version="0.0.categ_v1", 
                                                          predict_choice_method=predict_choice_method,
                                                          dir_name="single_partition/",
                                                          depth=depth,
                                                          repeats_n=3)

            except exceptions.TagsLengthNeedsToBeGreaterThanK:
                print("TagsLengthNeedsToBeGreaterThanK")



n: 1 - user_id: 6189
n: 2 - user_id: 5936
n: 3 - user_id: 6087
n: 4 - user_id: 5973
n: 5 - user_id: 6085
n: 6 - user_id: 6074
n: 7 - user_id: 6012
n: 8 - user_id: 5982
n: 9 - user_id: 5948
n: 10 - user_id: 5974
n: 11 - user_id: 6090
n: 12 - user_id: 6199
n: 13 - user_id: 6068
n: 14 - user_id: 6024
n: 15 - user_id: 5976
n: 16 - user_id: 6094
n: 17 - user_id: 5941
n: 18 - user_id: 5995
n: 19 - user_id: 5962
n: 20 - user_id: 6093
n: 21 - user_id: 6033
n: 22 - user_id: 6079
n: 23 - user_id: 6038
n: 24 - user_id: 6175
n: 25 - user_id: 6042
n: 26 - user_id: 5924
n: 27 - user_id: 6083
n: 28 - user_id: 6178
n: 29 - user_id: 5958
n: 30 - user_id: 6086
n: 31 - user_id: 5945
n: 32 - user_id: 5925
n: 33 - user_id: 5935
n: 34 - user_id: 6172
n: 35 - user_id: 5955
n: 36 - user_id: 6073
n: 37 - user_id: 5980
n: 38 - user_id: 6010
n: 39 - user_id: 5993
n: 40 - user_id: 6037
n: 41 - user_id: 5979
n: 42 - user_id: 5966
n: 43 - user_id: 5985
n: 44 - user_id: 5967
n: 45 - user_id: 5970
n: 46 - user_id: 61

In [6]:
skipped_users

[]

## Markov

#### SINGLE PARTITION - Same train and test data for each same user

In [7]:
skipped_users = []

n=0
for user_id in users_categ_sequence.keys():
    n += 1
    print("n:", n, "-", "user_id:", user_id)

    if len( remove_list_elements(users_categ_sequence[user_id], elements=[[]]) ) < 8:
        skipped_users.append(user_id)
        continue    

    try:
        evaluation_markov_single_partition_light_mem(tags_sequence=users_categ_sequence[user_id], 
                                                      user_id=user_id, 
                                                      input_data_version="0.0.categ_v1", 
                                                      dir_name="single_partition/",
                                                      repeats_n=3,
                                                      is_distributive=False)
        
    except exceptions.TagsLengthNeedsToBeGreaterThanK:
        print("TagsLengthNeedsToBeGreaterThanK")



n: 1 - user_id: 6189
n: 2 - user_id: 5936
n: 3 - user_id: 6087
n: 4 - user_id: 5973
n: 5 - user_id: 6085
n: 6 - user_id: 6074
n: 7 - user_id: 6012
n: 8 - user_id: 5982
n: 9 - user_id: 5948
n: 10 - user_id: 5974
n: 11 - user_id: 6090
n: 12 - user_id: 6199
n: 13 - user_id: 6068
n: 14 - user_id: 6024
n: 15 - user_id: 5976
n: 16 - user_id: 6094
n: 17 - user_id: 5941
n: 18 - user_id: 5995
n: 19 - user_id: 5962
n: 20 - user_id: 6093
n: 21 - user_id: 6033
n: 22 - user_id: 6079
n: 23 - user_id: 6038
n: 24 - user_id: 6175
n: 25 - user_id: 6042
n: 26 - user_id: 5924
n: 27 - user_id: 6083
n: 28 - user_id: 6178
n: 29 - user_id: 5958
n: 30 - user_id: 6086
n: 31 - user_id: 5945
n: 32 - user_id: 5925
n: 33 - user_id: 5935
n: 34 - user_id: 6172
n: 35 - user_id: 5955
n: 36 - user_id: 6073
n: 37 - user_id: 5980
n: 38 - user_id: 6010
n: 39 - user_id: 5993
n: 40 - user_id: 6037
n: 41 - user_id: 5979
n: 42 - user_id: 5966
n: 43 - user_id: 5985
n: 44 - user_id: 5967
n: 45 - user_id: 5970
n: 46 - user_id: 61

#### CLUSTER - Test with one user, train with the rest of the cluster

In [15]:
knn_clutsers = pd.read_csv("outputs/sr_time_quartiles/knn_freq_categ.csv")
del knn_clutsers["0"]
knn_clutsers.head()

,1,2,3,4,user_id
0,6015,5988,5947,5925,5924
1,5947,6038,5940,6072,5925
2,6038,6104,6014,6072,5927
3,6001,5967,6167,6042,5928
4,5948,6001,5965,6072,5935


In [ ]:
# cluster_tag_sequences = {}

# for cluster in clusters["cluster"].drop_duplicates():
#     cluster_tag_sequences[cluster] = {}

#     cluster_data = clusters[clusters["cluster"] == cluster]

#     for user_id in cluster_data["user_id"]:
    
#         try:
            
#             cluster_tag_sequences[cluster][str(user_id)] = users_categ_sequence[str(user_id)]
    
#         except KeyError:
#             print("user id not used:", user_id)
#             pass


In [ ]:
# n=0
# for cluster in cluster_tag_sequences.keys():
#     n += 1
#     cluster_data = cluster_tag_sequences[cluster]
    
#     for user_id in cluster_data.keys():
#         print(user_id)
    
#         try:
#             evaluation_markov_cluster_light_mem(cluster=cluster_data, 
#                                                 test_user_id=user_id, 
#                                                 input_data_version="0.0.categ_v1",
#                                                 cluster_version="dummy_cluster",
#                                                 dir_name="single_partition/", 
#                                                 repeats_n=3, 
#                                                 save_result=True)
            
#         except exceptions.ClusterSizeInadequate:
#             print("Small cluster size:", len(cluster_data))


#### K FOLD - Train and Test with the same user

In [ ]:
skipped_users = []

k=4

n=0
for user_id in users_categ_sequence.keys():
    n += 1
    print("n:", n, "-", "user_id:", user_id)

    if len( remove_list_elements(tags_sequence, elements=[[]]) ) < 8:
        skipped_users.append(user_id)
        continue    

    try:
        evaluation_markov_k_fold_light_mem(users_tags_sequence[user_id],
                                           input_data_version="0.0", 
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=False)
        
        evaluation_markov_k_fold_light_mem(users_tags_sequence[user_id], 
                                           input_data_version="0.0", 
                                           random_dummy_mode="dummy",
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=False)
        
        evaluation_markov_k_fold_light_mem(users_tags_sequence[user_id], 
                                           input_data_version="0.0.d", 
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=True)
        
        evaluation_markov_k_fold_light_mem(users_tags_sequence[user_id], 
                                           input_data_version="0.0.d", 
                                           random_dummy_mode="dummy",
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=True)

        evaluation_markov_k_fold_light_mem(users_categ_sequence[user_id], 
                                           input_data_version="0.0.categ_v1", 
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=False)

        evaluation_markov_k_fold_light_mem(users_categ_sequence[user_id], 
                                           input_data_version="0.0.categ_v1", 
                                           random_dummy_mode="dummy",
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=False)


    except exceptions.TagsLengthNeedsToBeGreaterThanK:
        print("TagsLengthNeedsToBeGreaterThanK")



In [ ]:
skipped_users

#### Test with one user, train with all the rest of them

In [10]:
# evaluation_markov_all_users_vs_one_light_mem(users_tags_sequence, 
#                                       dir_name="results",
#                                       input_data_version="0.0", 
#                                       is_distributive=False)

# evaluation_markov_all_users_vs_one_light_mem(users_tags_sequence, 
#                                       dir_name="results",
#                                       random_dummy_mode="dummy", 
#                                       input_data_version="0.0", 
#                                       is_distributive=False)


# evaluation_markov_all_users_vs_one_light_mem(users_tags_sequence, 
#                                       dir_name="results",
#                                       input_data_version="0.0.d", 
#                                       is_distributive=True)

# evaluation_markov_all_users_vs_one_light_mem(users_tags_sequence, 
#                                       dir_name="results",
#                                       random_dummy_mode="dummy", 
#                                       input_data_version="0.0.d", 
#                                       is_distributive=True)


evaluation_markov_all_users_vs_one_light_mem(users_categ_sequence, 
                                      dir_name="single_partition",
                                      input_data_version="0.0.categ_v1", 
                                      is_distributive=False)

evaluation_markov_all_users_vs_one_light_mem(users_categ_sequence, 
                                      dir_name="single_partition",
                                      random_dummy_mode="dummy", 
                                      input_data_version="0.0.categ_v1", 
                                      is_distributive=False)

n: 1 - test_user: 6189
n: 2 - test_user: 5936
n: 3 - test_user: 6087
n: 4 - test_user: 5973
n: 5 - test_user: 6085
n: 6 - test_user: 6074
n: 7 - test_user: 6012
n: 8 - test_user: 5982
n: 9 - test_user: 5948
n: 10 - test_user: 5974
n: 11 - test_user: 6090
n: 12 - test_user: 6199
n: 13 - test_user: 6068
n: 14 - test_user: 6024
n: 15 - test_user: 5976
n: 16 - test_user: 6094
n: 17 - test_user: 5941
n: 18 - test_user: 5995
n: 19 - test_user: 5962
n: 20 - test_user: 6093
n: 21 - test_user: 6033
n: 22 - test_user: 6079
n: 23 - test_user: 6038
n: 24 - test_user: 6175
n: 25 - test_user: 6042
n: 26 - test_user: 5924
n: 27 - test_user: 6083
n: 28 - test_user: 6178
n: 29 - test_user: 5958
n: 30 - test_user: 6086
n: 31 - test_user: 5945
n: 32 - test_user: 5925
n: 33 - test_user: 5935
n: 34 - test_user: 6172
n: 35 - test_user: 5955
n: 36 - test_user: 6073
n: 37 - test_user: 5980
n: 38 - test_user: 6010
n: 39 - test_user: 5993
n: 40 - test_user: 6037
n: 41 - test_user: 5979
n: 42 - test_user: 5966
n